### In this file the notes in dataframes removed because they are long. EMMA has problem in terms of reading long string as a note. Instead, the excel file names are used as notes contents.

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [7]:
pwd

'C:\\Users\\ssabahi\\Desktop\\EMMA_Final\\EMMA_Soil_2018-2019\\EMMA_Water_2018-2019'

In [8]:
import glob
# file_names = sorted(glob.glob('*.xlsx'))
file_names = (glob.glob('*.xlsx'))


In [9]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [10]:
file_names

['3008 Water chem results 20018+2019.xlsx',
 '3008 Water chem results 2018+2019_field.xlsx',
 '3008 Water chem results 2019-2467 Rev3B.xlsx',
 '3008 Water chem Results 2019-7104 RevA.xlsx',
 '3008_19FALL_WS chem results 2019-14330_Final.xlsx',
 '3008_19FALL_WS-field results RevA_Final.xlsx',
 '3008_19SPRI_WS -field RevA.xlsx',
 '3008_19summ_WS results 2019-11059.xlsx']

## Columns needed for future EMMA dataframes


In [11]:
emma_cols = ['Site_Name', 'Media_Name', 'Lab_Track_ID', 'Location_Name', 'Sample_ID',
       'Lab_Sample_ID', 'Duplicate_Source_ID', 'Duplicate_Lab_Source_ID',
       'Is_Travel_Blank', 'Is_Field_Blank', 'Depth_Start_M', 'Depth_End_M',
       'Diluted', 'Sample_DateTime', 'Analysis_DateTime', 'Analysis_Method',
       'Parameter_ID', 'Sample_Value', 'Less_Than_DL', 'Detection_Limit',
       'Units', 'Spike_Recovery', 'Species', 'Sample_Treatment',
       'Soil_Texture', 'Notes']

# len(emma_cols) = 26

## Columns needed to keep  based on LAB_NAME_CODE: SRC

In [12]:
col_ls = ['FACILITY_NAME','MATRIX_DESC','SYS_LOC_CODE','SAMPLE_NAME','LAB_SAMPLE_ID','START_DEPTH', 'END_DEPTH',
          'SAMPLE_DATE','ANALYSIS_DATE', 'METHOD_ANALYTE_GROUP_DESC','CHEMICAL_NAME','REPORT_RESULT_VALUE',
          'DETECT_FLAG', 'REPORTING_DETECTION_LIMIT' ,'REPORT_RESULT_UNIT', 'NEW_NOTE']

# len(col_ls) = 16
# 10 more columns are needed to complete the EMMA format

### Using excel file names as notes content

In [13]:
data = []
for i in range(0,len(file_names)):
    data.append(pd.read_excel(file_names[i]))
    data[i]['NEW_NOTE'] = file_names[i]        

### Columns in different excel files are not the same and we need to add or remove some of them based on the col_ls list (defined above)

### The status of the current dataframe columns compare to the list col_ls + the current dataframes dimension

In [14]:
col_df = pd.DataFrame(columns=['Excel Files','Columns Needed','Dimension'], index=list(range(1,len(file_names)+1)))

In [15]:
for i,j in enumerate(file_names):
    col_df.iloc[i]['Excel Files'] = j
    col_df.iloc[i]['Columns Needed'] = list(set(col_ls) - set(list(data[i].columns)))
    col_df.iloc[i]['Dimension'] = data[i].shape
    
col_df

,Excel Files,Columns Needed,Dimension
1,3008 Water chem results 20018+2019.xlsx,[],"(5516, 43)"
2,3008 Water chem results 2018+2019_field.xlsx,[],"(4333, 43)"
3,3008 Water chem results 2019-2467 Rev3B.xlsx,[],"(2202, 38)"
4,3008 Water chem Results 2019-7104 RevA.xlsx,[],"(2473, 36)"
5,3008_19FALL_WS chem results 2019-14330_Final.xlsx,[],"(2610, 37)"
6,3008_19FALL_WS-field results RevA_Final.xlsx,"[REPORTING_DETECTION_LIMIT, LAB_SAMPLE_ID]","(1265, 30)"
7,3008_19SPRI_WS -field RevA.xlsx,"[REPORTING_DETECTION_LIMIT, LAB_SAMPLE_ID]","(1349, 29)"
8,3008_19summ_WS results 2019-11059.xlsx,[],"(2915, 36)"


### The above tables shows that dataframes in row 6 and 7 need 2 more columns LAB_SAMPLE_ID and REPORTING_DETECTION_LIMIT

### We need to be careful about the dataframes in  row 6 and 7 in the table above when we remove the duplicates

### In the next block we add or remove some columns to make the same columns for all the dataframes 

In [16]:
for i in range(0,len(file_names)):

    if set(col_ls).issubset(list(data[i].columns)):
        data[i] = data[i][col_ls]
    else:
        data[i] = pd.concat([data[i] , pd.DataFrame(columns=list(set(col_ls) - set(list(data[i].columns))) )],sort=False) 
        data[i] = data[i][col_ls]  
        

### Checking the dimension of modified dataframes

In [17]:
data_info_df = pd.DataFrame(columns=['Excel Files','Dimension'], index=list(range(1,len(file_names)+1)))

In [18]:
for i,j in enumerate(file_names):
    data_info_df.iloc[i]['Excel Files'] = j
    data_info_df.iloc[i]['Dimension'] = data[i].shape                           

In [19]:
data_info_df

,Excel Files,Dimension
1,3008 Water chem results 20018+2019.xlsx,"(5516, 16)"
2,3008 Water chem results 2018+2019_field.xlsx,"(4333, 16)"
3,3008 Water chem results 2019-2467 Rev3B.xlsx,"(2202, 16)"
4,3008 Water chem Results 2019-7104 RevA.xlsx,"(2473, 16)"
5,3008_19FALL_WS chem results 2019-14330_Final.xlsx,"(2610, 16)"
6,3008_19FALL_WS-field results RevA_Final.xlsx,"(1265, 16)"
7,3008_19SPRI_WS -field RevA.xlsx,"(1349, 16)"
8,3008_19summ_WS results 2019-11059.xlsx,"(2915, 16)"


### Now we need to concatinate the excel files and make one master file

In [20]:
master_df = pd.DataFrame(columns=col_ls)
for i in range(0,len(file_names)):
    master_df = pd.concat([master_df,data[i]],ignore_index=True)

In [21]:
master_df.shape

(22663, 16)

In [22]:
master_df.head()

,FACILITY_NAME,MATRIX_DESC,SYS_LOC_CODE,SAMPLE_NAME,LAB_SAMPLE_ID,START_DEPTH,END_DEPTH,SAMPLE_DATE,ANALYSIS_DATE,METHOD_ANALYTE_GROUP_DESC,CHEMICAL_NAME,REPORT_RESULT_VALUE,DETECT_FLAG,REPORTING_DETECTION_LIMIT,REPORT_RESULT_UNIT,NEW_NOTE
0,Patterson Lake,Surface Water,CLEARV1,3008 FB Water Sample,2018020699,0,0,2018-05-21 13:48:00,2018-06-08 08:31:36,Physical Properties,"Hardness, Total",1.00,N,1,mg/L,3008 Water chem results 20018+2019.xlsx
1,Patterson Lake,Surface Water,CLEARV1,3008 FB Water Sample,2018020699,0,0,2018-05-21 13:48:00,2018-06-07 09:08:10,Metals,Mercury,1.00,N,1,ng/L,3008 Water chem results 20018+2019.xlsx
2,Patterson Lake,Surface Water,CLEARV1,3008 FB Water Sample,2018020699,0,0,2018-05-21 13:48:00,2018-06-07 16:01:56,Nutrients,Ammonia (as N),0.01,N,0.01,mg/L,3008 Water chem results 20018+2019.xlsx
3,Patterson Lake,Surface Water,CLEARV1,3008 FB Water Sample,2018020699,0,0,2018-05-21 13:48:00,2018-06-10 14:08:14,Nutrients,"Nitrogen, Total Kjeldahl",0.05,N,0.05,mg/L,3008 Water chem results 20018+2019.xlsx
4,Patterson Lake,Surface Water,CLEARV1,3008 FB Water Sample,2018020699,0,0,2018-05-21 13:48:00,2018-06-06 16:31:03,Nutrients,Nitrate (NO3),0.04,N,0.04,mg/L,3008 Water chem results 20018+2019.xlsx


### To check whether we are using the right data for surface water

In [23]:
master_df['MATRIX_DESC'].nunique()

1

In [24]:
# To check that all excel files are belong to surface water

In [25]:
master_df['MATRIX_DESC'].value_counts() 

Surface Water    22663
Name: MATRIX_DESC, dtype: int64

### Check the duplicate rows in the masterfile

### Create 3 different dataframes to find how duplicate rows are changing

In [26]:
master_temp1_df = master_df.drop(columns=['NEW_NOTE'])
master_temp2_df = master_df.drop(columns=['LAB_SAMPLE_ID','REPORTING_DETECTION_LIMIT'])   
master_temp3_df = master_df.drop(columns=['LAB_SAMPLE_ID','REPORTING_DETECTION_LIMIT','NEW_NOTE'])  

In [27]:
print(f'The shape of "master_temp1_df" is {master_temp1_df[master_temp1_df.duplicated()].shape}')
print(f'The shape of "master_temp2_df" is {master_temp2_df[master_temp2_df.duplicated()].shape}')
print(f'The shape of "master_temp3_df" is {master_temp3_df[master_temp3_df.duplicated()].shape}')
print(f'The shape of "master_df" is {master_df[master_df.duplicated()].shape}')

The shape of "master_temp1_df" is (2202, 15)
The shape of "master_temp2_df" is (0, 14)
The shape of "master_temp3_df" is (2202, 13)
The shape of "master_df" is (0, 16)


#### Comparing master_temp2_df and original master_df tells us that the duplicates are not in the dataframes related to rows 6 and 7 (see two table above)
### Therefrore, the duplicates needs to be removed after we drop the column NEW_NOTE 

In [28]:
master_df[master_df.drop(columns=['NEW_NOTE']).duplicated()].shape

(2202, 16)

### Frist remove the column "NEW_NOTES", save it in a new dataframe and then remove the duplicates. We need to add the remaining notes later to the modifed master dafaframe.

In [29]:
note_df = pd.DataFrame(master_df['NEW_NOTE'])
note_df.head(1)

,NEW_NOTE
0,3008 Water chem results 20018+2019.xlsx


In [30]:
master_df.drop(columns=['NEW_NOTE'],inplace=True)

In [31]:
master_df.drop_duplicates(inplace=True) 

In [32]:
master_df.shape

(20461, 15)

### We need to remove the same columns in note_df

In [33]:
set(note_df.index)-set(master_df.index)

{9849,
 9850,
 9851,
 9852,
 9853,
 9854,
 9855,
 9856,
 9857,
 9858,
 9859,
 9860,
 9861,
 9862,
 9863,
 9864,
 9865,
 9866,
 9867,
 9868,
 9869,
 9870,
 9871,
 9872,
 9873,
 9874,
 9875,
 9876,
 9877,
 9878,
 9879,
 9880,
 9881,
 9882,
 9883,
 9884,
 9885,
 9886,
 9887,
 9888,
 9889,
 9890,
 9891,
 9892,
 9893,
 9894,
 9895,
 9896,
 9897,
 9898,
 9899,
 9900,
 9901,
 9902,
 9903,
 9904,
 9905,
 9906,
 9907,
 9908,
 9909,
 9910,
 9911,
 9912,
 9913,
 9914,
 9915,
 9916,
 9917,
 9918,
 9919,
 9920,
 9921,
 9922,
 9923,
 9924,
 9925,
 9926,
 9927,
 9928,
 9929,
 9930,
 9931,
 9932,
 9933,
 9934,
 9935,
 9936,
 9937,
 9938,
 9939,
 9940,
 9941,
 9942,
 9943,
 9944,
 9945,
 9946,
 9947,
 9948,
 9949,
 9950,
 9951,
 9952,
 9953,
 9954,
 9955,
 9956,
 9957,
 9958,
 9959,
 9960,
 9961,
 9962,
 9963,
 9964,
 9965,
 9966,
 9967,
 9968,
 9969,
 9970,
 9971,
 9972,
 9973,
 9974,
 9975,
 9976,
 9977,
 9978,
 9979,
 9980,
 9981,
 9982,
 9983,
 9984,
 9985,
 9986,
 9987,
 9988,
 9989,
 9990,
 9991,

In [34]:
note_df.drop(index=list(set(note_df.index)-set(master_df.index)), inplace=True)

In [35]:
note_df.head()

,NEW_NOTE
0,3008 Water chem results 20018+2019.xlsx
1,3008 Water chem results 20018+2019.xlsx
2,3008 Water chem results 20018+2019.xlsx
3,3008 Water chem results 20018+2019.xlsx
4,3008 Water chem results 20018+2019.xlsx


In [36]:
note_df.shape

(20461, 1)

### Now we need to concatinate the master_df and note_df  

In [37]:
master_df = pd.concat([ master_df , note_df ],axis=1,sort=False)

In [38]:
master_df.head()

,FACILITY_NAME,MATRIX_DESC,SYS_LOC_CODE,SAMPLE_NAME,LAB_SAMPLE_ID,START_DEPTH,END_DEPTH,SAMPLE_DATE,ANALYSIS_DATE,METHOD_ANALYTE_GROUP_DESC,CHEMICAL_NAME,REPORT_RESULT_VALUE,DETECT_FLAG,REPORTING_DETECTION_LIMIT,REPORT_RESULT_UNIT,NEW_NOTE
0,Patterson Lake,Surface Water,CLEARV1,3008 FB Water Sample,2018020699,0,0,2018-05-21 13:48:00,2018-06-08 08:31:36,Physical Properties,"Hardness, Total",1.00,N,1,mg/L,3008 Water chem results 20018+2019.xlsx
1,Patterson Lake,Surface Water,CLEARV1,3008 FB Water Sample,2018020699,0,0,2018-05-21 13:48:00,2018-06-07 09:08:10,Metals,Mercury,1.00,N,1,ng/L,3008 Water chem results 20018+2019.xlsx
2,Patterson Lake,Surface Water,CLEARV1,3008 FB Water Sample,2018020699,0,0,2018-05-21 13:48:00,2018-06-07 16:01:56,Nutrients,Ammonia (as N),0.01,N,0.01,mg/L,3008 Water chem results 20018+2019.xlsx
3,Patterson Lake,Surface Water,CLEARV1,3008 FB Water Sample,2018020699,0,0,2018-05-21 13:48:00,2018-06-10 14:08:14,Nutrients,"Nitrogen, Total Kjeldahl",0.05,N,0.05,mg/L,3008 Water chem results 20018+2019.xlsx
4,Patterson Lake,Surface Water,CLEARV1,3008 FB Water Sample,2018020699,0,0,2018-05-21 13:48:00,2018-06-06 16:31:03,Nutrients,Nitrate (NO3),0.04,N,0.04,mg/L,3008 Water chem results 20018+2019.xlsx


In [47]:
master_df[master_df['SYS_LOC_CODE']=='MIRRRV']

,FACILITY_NAME,MATRIX_DESC,SYS_LOC_CODE,SAMPLE_NAME,LAB_SAMPLE_ID,START_DEPTH,END_DEPTH,SAMPLE_DATE,ANALYSIS_DATE,METHOD_ANALYTE_GROUP_DESC,CHEMICAL_NAME,REPORT_RESULT_VALUE,DETECT_FLAG,REPORTING_DETECTION_LIMIT,REPORT_RESULT_UNIT,NEW_NOTE
1832,Patterson Lake,Surface Water,MIRRRV,Mirror River Water Sample,2018032831,0,0,2018-08-04 09:45:00,2018-08-24 08:46:30,Physical Properties,"Hardness, Total",10.00000,Y,1,mg/L,3008 Water chem results 20018+2019.xlsx
1833,Patterson Lake,Surface Water,MIRRRV,Mirror River Water Sample,2018032831,0,0,2018-08-04 09:45:00,2018-08-23 14:27:44,Metals,Mercury,1.00000,N,1,ng/L,3008 Water chem results 20018+2019.xlsx
1834,Patterson Lake,Surface Water,MIRRRV,Mirror River Water Sample,2018032831,0,0,2018-08-04 09:45:00,2018-08-22 11:16:31,Nutrients,Ammonia (as N),0.01000,N,0.01,mg/L,3008 Water chem results 20018+2019.xlsx
1835,Patterson Lake,Surface Water,MIRRRV,Mirror River Water Sample,2018032831,0,0,2018-08-04 09:45:00,2018-08-23 14:51:17,Nutrients,"Nitrogen, Total Kjeldahl",0.34000,Y,0.05,mg/L,3008 Water chem results 20018+2019.xlsx
1836,Patterson Lake,Surface Water,MIRRRV,Mirror River Water Sample,2018032831,0,0,2018-08-04 09:45:00,2018-08-22 11:17:21,Nutrients,Nitrate (as N),0.01000,N,0.01,mg/L,3008 Water chem results 20018+2019.xlsx
1837,Patterson Lake,Surface Water,MIRRRV,Mirror River Water Sample,2018032831,0,0,2018-08-04 09:45:00,2018-09-07 08:41:34,Radionuclides,Polonium-210,0.00500,N,0.005,Bq/L,3008 Water chem results 20018+2019.xlsx
1838,Patterson Lake,Surface Water,MIRRRV,Mirror River Water Sample,2018032831,0,0,2018-08-04 09:45:00,2018-09-07 15:23:57,Radionuclides,Radium-226,0.00500,N,0.005,Bq/L,3008 Water chem results 20018+2019.xlsx
1839,Patterson Lake,Surface Water,MIRRRV,Mirror River Water Sample,2018032831,0,0,2018-08-04 09:45:00,2018-09-13 08:07:27,Radionuclides,Thorium-230,0.01000,N,0.01,Bq/L,3008 Water chem results 20018+2019.xlsx
1840,Patterson Lake,Surface Water,MIRRRV,Mirror River Water Sample,2018032831,0,0,2018-08-04 09:45:00,2018-09-07 09:01:19,Radionuclides,Lead-210,0.02000,N,0.02,Bq/L,3008 Water chem results 20018+2019.xlsx
1841,Patterson Lake,Surface Water,MIRRRV,Mirror River Water Sample,2018032831,0,0,2018-08-04 09:45:00,2018-08-24 08:46:30,Physical Properties,Sum of ions,20.00000,Y,1,mg/L,3008 Water chem results 20018+2019.xlsx


In [39]:
master_df['SYS_LOC_CODE'].unique()

array(['CLEARV1', 'BEETCK', 'BEETLK1', 'BROALK1', 'FORRLK1', 'FORRLK2',
       'JEDCK', 'LAKEG1', 'LAKEH1', 'LLOYLKI', 'LLOYLKO', 'NAOMLK',
       'PATTLKE1', 'PATTLKN1', 'PATTLKNE1', 'WARNRP', 'CLEARV_MF1',
       'CLEARV_MF2', 'LAKEG2', 'LAKEH2', 'MIRRRV', 'BROALK2', 'HODGLK1',
       'LAKED', 'LAKEJ', 'NAOMLK-1', 'LAKEI', 'CLEARV_MF3'], dtype=object)

In [40]:
master_df[master_df['SYS_LOC_CODE']=='LAKEI']['NEW_NOTE']

9293    3008 Water chem results 2018+2019_field.xlsx
9294    3008 Water chem results 2018+2019_field.xlsx
9295    3008 Water chem results 2018+2019_field.xlsx
9296    3008 Water chem results 2018+2019_field.xlsx
9297    3008 Water chem results 2018+2019_field.xlsx
9298    3008 Water chem results 2018+2019_field.xlsx
9299    3008 Water chem results 2018+2019_field.xlsx
9300    3008 Water chem results 2018+2019_field.xlsx
9301    3008 Water chem results 2018+2019_field.xlsx
9302    3008 Water chem results 2018+2019_field.xlsx
9303    3008 Water chem results 2018+2019_field.xlsx
9304    3008 Water chem results 2018+2019_field.xlsx
9305    3008 Water chem results 2018+2019_field.xlsx
9306    3008 Water chem results 2018+2019_field.xlsx
9307    3008 Water chem results 2018+2019_field.xlsx
9308    3008 Water chem results 2018+2019_field.xlsx
9309    3008 Water chem results 2018+2019_field.xlsx
9310    3008 Water chem results 2018+2019_field.xlsx
9311    3008 Water chem results 2018+2019_fiel

In [41]:
master_df.shape

(20461, 16)

### Preparing the final format of the EMMA file

In [42]:
master_df.columns

Index(['FACILITY_NAME', 'MATRIX_DESC', 'SYS_LOC_CODE', 'SAMPLE_NAME', 'LAB_SAMPLE_ID', 'START_DEPTH', 'END_DEPTH', 'SAMPLE_DATE', 'ANALYSIS_DATE', 'METHOD_ANALYTE_GROUP_DESC', 'CHEMICAL_NAME', 'REPORT_RESULT_VALUE', 'DETECT_FLAG', 'REPORTING_DETECTION_LIMIT', 'REPORT_RESULT_UNIT', 'NEW_NOTE'], dtype='object')

In [43]:
emma_cols

['Site_Name',
 'Media_Name',
 'Lab_Track_ID',
 'Location_Name',
 'Sample_ID',
 'Lab_Sample_ID',
 'Duplicate_Source_ID',
 'Duplicate_Lab_Source_ID',
 'Is_Travel_Blank',
 'Is_Field_Blank',
 'Depth_Start_M',
 'Depth_End_M',
 'Diluted',
 'Sample_DateTime',
 'Analysis_DateTime',
 'Analysis_Method',
 'Parameter_ID',
 'Sample_Value',
 'Less_Than_DL',
 'Detection_Limit',
 'Units',
 'Spike_Recovery',
 'Species',
 'Sample_Treatment',
 'Soil_Texture',
 'Notes']

### We need to rename the master_df columns name based on emma_cols

In [45]:
master_df.rename(columns = {'FACILITY_NAME': 'Site_Name',                'MATRIX_DESC':'Media_Name',    
                            'SYS_LOC_CODE':'Location_Name',              'SAMPLE_NAME':'Sample_ID',   
                            'LAB_SAMPLE_ID':'Lab_Sample_ID',             'START_DEPTH':'Depth_Start_M', 
                            'END_DEPTH':'Depth_End_M',                   'SAMPLE_DATE': 'Sample_DateTime',
                            'ANALYSIS_DATE':'Analysis_DateTime',         'METHOD_ANALYTE_GROUP_DESC':'Analysis_Method',            
                            'CHEMICAL_NAME':'Parameter_ID',              'REPORT_RESULT_VALUE':'Sample_Value',      
                            'DETECT_FLAG':'Less_Than_DL',                'REPORTING_DETECTION_LIMIT':'Detection_Limit',                 
                            'REPORT_RESULT_UNIT':'Units',                'NEW_NOTE':'Notes'
                                 }, inplace=True)

In [46]:
rem_EMMA_ls = ['Lab_Track_ID','Duplicate_Source_ID','Duplicate_Lab_Source_ID','Is_Travel_Blank',
 'Is_Field_Blank','Diluted', 'Spike_Recovery','Species','Sample_Treatment','Soil_Texture','Notes' ]

In [47]:
master_df = pd.concat([ master_df , pd.DataFrame(columns=rem_EMMA_ls) ],sort=False)

In [48]:
master_df = master_df[emma_cols] 

In [49]:
master_df.head()

,Site_Name,Media_Name,Lab_Track_ID,Location_Name,Sample_ID,Lab_Sample_ID,Duplicate_Source_ID,Duplicate_Lab_Source_ID,Is_Travel_Blank,Is_Field_Blank,Depth_Start_M,Depth_End_M,Diluted,Sample_DateTime,Analysis_DateTime,Analysis_Method,Parameter_ID,Sample_Value,Less_Than_DL,Detection_Limit,Units,Spike_Recovery,Species,Sample_Treatment,Soil_Texture,Notes
0,Patterson Lake,Surface Water,NaN,CLEARV1,3008 FB Water Sample,2018020699,NaN,NaN,NaN,NaN,0,0,NaN,2018-05-21 13:48:00,2018-06-08 08:31:36,Physical Properties,"Hardness, Total",1.00,N,1,mg/L,NaN,NaN,NaN,NaN,3008 Water chem results 20018+2019.xlsx
1,Patterson Lake,Surface Water,NaN,CLEARV1,3008 FB Water Sample,2018020699,NaN,NaN,NaN,NaN,0,0,NaN,2018-05-21 13:48:00,2018-06-07 09:08:10,Metals,Mercury,1.00,N,1,ng/L,NaN,NaN,NaN,NaN,3008 Water chem results 20018+2019.xlsx
2,Patterson Lake,Surface Water,NaN,CLEARV1,3008 FB Water Sample,2018020699,NaN,NaN,NaN,NaN,0,0,NaN,2018-05-21 13:48:00,2018-06-07 16:01:56,Nutrients,Ammonia (as N),0.01,N,0.01,mg/L,NaN,NaN,NaN,NaN,3008 Water chem results 20018+2019.xlsx
3,Patterson Lake,Surface Water,NaN,CLEARV1,3008 FB Water Sample,2018020699,NaN,NaN,NaN,NaN,0,0,NaN,2018-05-21 13:48:00,2018-06-10 14:08:14,Nutrients,"Nitrogen, Total Kjeldahl",0.05,N,0.05,mg/L,NaN,NaN,NaN,NaN,3008 Water chem results 20018+2019.xlsx
4,Patterson Lake,Surface Water,NaN,CLEARV1,3008 FB Water Sample,2018020699,NaN,NaN,NaN,NaN,0,0,NaN,2018-05-21 13:48:00,2018-06-06 16:31:03,Nutrients,Nitrate (NO3),0.04,N,0.04,mg/L,NaN,NaN,NaN,NaN,3008 Water chem results 20018+2019.xlsx


### Need to modify the Less_Than_DL column

In [50]:
master_df['Less_Than_DL'].value_counts() 

Y    13038
N     7423
Name: Less_Than_DL, dtype: int64

In [51]:
master_df['Less_Than_DL'] = master_df['Less_Than_DL'].apply(lambda row: True if row=='N' else False)

In [52]:
master_df.head()

,Site_Name,Media_Name,Lab_Track_ID,Location_Name,Sample_ID,Lab_Sample_ID,Duplicate_Source_ID,Duplicate_Lab_Source_ID,Is_Travel_Blank,Is_Field_Blank,Depth_Start_M,Depth_End_M,Diluted,Sample_DateTime,Analysis_DateTime,Analysis_Method,Parameter_ID,Sample_Value,Less_Than_DL,Detection_Limit,Units,Spike_Recovery,Species,Sample_Treatment,Soil_Texture,Notes
0,Patterson Lake,Surface Water,NaN,CLEARV1,3008 FB Water Sample,2018020699,NaN,NaN,NaN,NaN,0,0,NaN,2018-05-21 13:48:00,2018-06-08 08:31:36,Physical Properties,"Hardness, Total",1.00,True,1,mg/L,NaN,NaN,NaN,NaN,3008 Water chem results 20018+2019.xlsx
1,Patterson Lake,Surface Water,NaN,CLEARV1,3008 FB Water Sample,2018020699,NaN,NaN,NaN,NaN,0,0,NaN,2018-05-21 13:48:00,2018-06-07 09:08:10,Metals,Mercury,1.00,True,1,ng/L,NaN,NaN,NaN,NaN,3008 Water chem results 20018+2019.xlsx
2,Patterson Lake,Surface Water,NaN,CLEARV1,3008 FB Water Sample,2018020699,NaN,NaN,NaN,NaN,0,0,NaN,2018-05-21 13:48:00,2018-06-07 16:01:56,Nutrients,Ammonia (as N),0.01,True,0.01,mg/L,NaN,NaN,NaN,NaN,3008 Water chem results 20018+2019.xlsx
3,Patterson Lake,Surface Water,NaN,CLEARV1,3008 FB Water Sample,2018020699,NaN,NaN,NaN,NaN,0,0,NaN,2018-05-21 13:48:00,2018-06-10 14:08:14,Nutrients,"Nitrogen, Total Kjeldahl",0.05,True,0.05,mg/L,NaN,NaN,NaN,NaN,3008 Water chem results 20018+2019.xlsx
4,Patterson Lake,Surface Water,NaN,CLEARV1,3008 FB Water Sample,2018020699,NaN,NaN,NaN,NaN,0,0,NaN,2018-05-21 13:48:00,2018-06-06 16:31:03,Nutrients,Nitrate (NO3),0.04,True,0.04,mg/L,NaN,NaN,NaN,NaN,3008 Water chem results 20018+2019.xlsx


In [53]:
master_df['Less_Than_DL'].value_counts() 

False    13038
True      7423
Name: Less_Than_DL, dtype: int64

In [87]:
master_df['Parameter_ID'].unique()

array(['Hardness, Total', 'Mercury', 'Ammonia (as N)',
       'Nitrogen, Total Kjeldahl', 'Nitrate (NO3)', 'Nitrate (as N)',
       'Radium-226', 'Polonium-210', 'Thorium-230', 'Lead-210',
       'Sum of ions', 'Turbidity', 'Alkalinity, Total (as CaCO3)',
       'Alkalinity, Phenolphthalein', 'Carbonate (as CO3)',
       'Bicarbonate (as HCO3)', 'Hydroxide (as OH)',
       'Specific Conductivity', 'Solids, Total dissolved',
       'Solids, Total suspended', 'Sulfate', 'Magnesium', 'Potassium',
       'Sodium', 'Calcium', 'Aluminum', 'Iron', 'Manganese', 'Titanium',
       'Beryllium', 'Boron', 'Chromium', 'Cobalt', 'Vanadium',
       'Phosphorus, Total', 'Lead', 'Molybdenum', 'Nickel', 'Silver',
       'Strontium', 'Thallium', 'Tin', 'Antimony', 'Arsenic', 'Barium',
       'Cadmium', 'Copper', 'Uranium', 'Zinc', 'Selenium', 'Chloride',
       'Fluoride', 'pH', 'Nitrate+Nitrite (as N) calc.',
       'Carbon, Total Organic', 'Carbon, Organic dissolved',
       'Ammonia (as N), dissolved'

In [70]:
ls1 = list(master_df[master_df['Parameter_ID'].str.contains('Oxygen')]['Sample_Value'])

In [71]:
ls1.remove(min(ls1))


In [86]:
[i for i in ls1 if i < 1]

[0.25,
 0.91,
 0.8,
 0.35,
 0.2,
 0.7,
 0.62,
 0.5,
 0.34,
 0.56,
 0.44,
 0.36,
 0.81,
 0.81,
 0.57,
 0.27,
 0.7,
 0.7]

### Saving the EMMA Excel File

In [54]:
out_path = "C:\\Users\\ssabahi\\Desktop\\EMMA_Water_2018-2019\\Final_Excel_File\\Water_EMMA_2018-2019.xlsx"
writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
master_df.to_excel(writer, sheet_name='Ready_For_EMMA', index=False)
writer.save()
